# EDA (Japanese)

M5コンペティション(Accuracy)のEDAノートブックです。コメントについては、日本語で書かせていただきます。

This is EDA notebook for M5 Accuracy competition. I'm sorry for writing comments in japanese.

WRMSSEの計算については、下記カーネルを参考にさせていただきました。

- This kernel is based on [for_Japanese_beginner(with WRMSSE in LGBM)](https://www.kaggle.com/girmdshinsei/for-japanese-beginner-with-wrmsse-in-lgbm)

コンペティションガイド[M5-Competitors-Guide_Final-1.pdf](https://mk0mcompetitiont8ake.kinstacdn.com/wp-content/uploads/2020/02/M5-Competitors-Guide_Final-1.pdf)に目を通していることを前提として書くため、各カラムの内容説明などは省略します。

In [ ]:
import datetime
import gc
import random
import warnings
from typing import List
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio; pio.renderers.default='notebook'
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

# from src.notebook_utils import reduce_memory_usage, merge_by_concat, show_basic_info
# from src.df_transformer import ReplaceBeforeFirstSell
# from src.plot_utils import PlotlyFigure

In [ ]:
def reduce_memory_usage(
    df: pd.DataFrame,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    受け取ったデータフレームについてそれぞれのカラムのデータ型を調べ、最小メモリ型に変換することで
    データフレームのメモリ使用量を削減する。
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and \
                   c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and \
                        c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and \
                        c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and \
                        c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and \
                   c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and \
                        c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(
            'Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'
            .format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def merge_by_concat(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    merge_on: List[str],
) -> pd.DataFrame:
    """
    型情報を失わずにデータフレームをマージする
    """
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1


def show_basic_info(df: pd.DataFrame):
    """
    データフレームの基本情報を表示する
    """
    df.info()
    display(df.head())
    display(df.tail())

In [ ]:
class ReplaceBeforeFirstSell(object):

    def __init__(self):
        pass

    def transform(
        self,
        df: pd.DataFrame,
        value_columns: List[str],
    ) -> pd.DataFrame:
        df_values = df[value_columns].values
        tmp = np.tile(np.arange(1, len(value_columns) + 1),
                      (df_values.shape[0], 1))
        tmp_values = ((df_values > 0) * tmp)
        start_no = np.min(np.where(
            tmp_values == 0, 9999, tmp_values), axis=1) - 1
        flag = np.dot(np.diag(1 / (start_no + 1)), tmp) < 1
        df_values = np.where(flag, np.nan, df_values)
        df[value_columns] = df_values
        return df

In [ ]:
class PlotlyFigure(object):
    """
    Provide application methods as adapter class of plotly
    """
    def __init__(self):
        pass

    def add_range_selector(
        self,
        fig: go.Figure,
    ) -> go.Figure:
        fig.update_xaxes(
            rangeselector=dict(
                buttons=list([
                    dict(count=12*7, label="12w",
                         step="day", stepmode="backward"),
                    dict(count=24*7, label="24w",
                         step="day", stepmode="backward"),
                    dict(count=36*7, label="36w",
                         step="day", stepmode="backward"),
                    dict(count=1, label="1y",
                         step="year", stepmode="backward"),
                    dict(count=2, label="2y",
                         step="year", stepmode="backward"),
                    dict(step="all")
                ])
            )
        )
        return fig

    def add_shape_region(
        self,
        fig: go.Figure,
        start_date: str,
        end_date: str,
        color: str = None,
    ) -> go.Figure:
        fig.add_shape(
            type='rect',
            xref='x',
            yref='paper',
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=color,
            opacity=0.5,
            layer='below',
            line_width=0,
        )
        return fig

    def format_annotation(
        self,
        fig: go.Figure,
        ax: float = 0,
        ay: float = -40,
        showarrow: bool = True,
        arrowhead: float = 7,
    ) -> go.Figure:
        fig.update_annotations(dict(
                    xref="x",
                    yref="y",
                    showarrow=showarrow,
                    arrowhead=arrowhead,
                    ax=ax,
                    ay=ay,
        ))
        return fig

In [ ]:
plotly_util = PlotlyFigure()
plt.style.use('ggplot')
random.seed(42)

In [ ]:
HISTORY_COUNTS = 1913
PRED_COUNTS = 28
NUM_ITEMS = 30490

HISTORY_COLUMNS = [f'd_{i + 1}' for i in range(HISTORY_COUNTS)]

## Load Dataset

コンペティションでは、3つのCSV形式のファイルで与えられます。

In [ ]:
calendar = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')
sales = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
prices = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')

### calendar.csv

calendar.csvには日付やイベントの情報が含まれます。

特徴的なのはsnap(Supplemental Nutrition Assistance Program：補充的栄養支援プログラム)関連のカラムである。SNAPとは、アメリカの低所得者向けの栄養補助プログラムであり、該当日においてSNAP購買が許可されるかどうかを表している。

- 参考PDF: https://www.maff.go.jp/primaff/kanko/project/attach/pdf/170900_28cr02_02.pdf

In [ ]:
show_basic_info(calendar)

### sales_train_validation.csv

sales_train_validation.csvには、商品関連のID(item_id, dept_id, etc)と売り上げの履歴(d_1～d_1913)が含まれます。

本コンペティションはvalidationとevaluationの2つのフェーズに分かれており、validationフェーズでは、d_1～d_1913のデータからd_1914～d_1941の28日間のデータを予測します。この期間は、リーダーボードにてスコアが表示されるため、こちらのスコアを参考にすることができます。

6月(evaluationフェーズ)にはd_1～d_1941のデータが与えられ、d_1942～d_1968の28日間のデータを予測します。evaluationフェーズでは、リーダーボードのスコアが隠されるので、参加者は過学習を避けるために、適切なCVの戦略を決める必要があります。(validationフェーズでも同様ですが)

- Note: 6月時点でd_1～d_1941のデータを含むsales_train_evaluation.csvが公開されます

In [ ]:
show_basic_info(sales)

### sell_prices.csv

sell_prices.csvには、店舗ごとの各商品の週単位での価格が格納されています。

In [ ]:
show_basic_info(prices)

## View raw data

本コンペティションにおいて予測したいのは「各店舗・商品全ての組み合わせについての28日先までの販売数」です。

その組み合わせは30,490通りあり、店舗や商品、イベント情報などの特徴をモデルに組み込んで良い予測ができるように学習する必要があります。

様々な集計を行うより先にまずは30490通りの中から、いくつかをランダムサンプリングしその販売数（ついでに価格)の推移を確認しましょう。

### Sales

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 10))
samples = random.sample(range(len(sales)), 10)

x = pd.to_datetime(calendar.iloc[:HISTORY_COUNTS, :]['date']).values
for ax, sample in zip(axes.flatten(), samples):
    y = sales[HISTORY_COLUMNS].loc[sample, :]
    ax.plot(x, y)
    ax.set_title(sales.loc[sample, 'id'])
    ax.set_xlabel('date')
    ax.set_ylabel('sales')

plt.tight_layout()
plt.show()

実行のたびに異なる商品の学習期間全体の販売数の推移が出力されます。販売数推移からは例えば以下のような特徴が読み取れます。

- 1日の販売数が少ない商品が多い (1日に1個も売れなかったり、売れても2, 3個だったりする)
- 急激に販売数が上がり元に戻る箇所が存在する(スパイク形状が存在する)
- 学習期間の途中から販売された商品が多数存在する。

### Prices

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 10))
samples = random.sample(range(len(sales)), 10)

for ax, sample in zip(axes.flatten(), samples):
    item = sales['item_id'].loc[sample]
    store = sales['store_id'].loc[sample]
    ax.plot(
        prices[(prices['item_id']==item) & (prices['store_id']==store)]['wm_yr_wk'],
        prices[(prices['item_id']==item) & (prices['store_id']==store)]['sell_price']
    )
    ax.set_title(f"Price of {sales['id'].loc[sample]}")
    ax.set_xlabel('week number')
    ax.set_ylabel('price')
    
plt.tight_layout()
plt.show()

- 値段が常に一定の商品もあれば、激しく変動する商品も存在する。

## Timeseries Aggregation

### Preprocessing

続いて時系列に着目して、様々な集計を実施します。

集計の前処理として、salesデータフレームに前処理を加えます。

下記はPDFの注記ですが、本コンペティションの評価指標となるRMSSEのスケール計算の際に使用されるデータは、初めて商品が販売された日以降のデータ
となります。

> Note that the denominator of RMSSE is computed only for the time-periods for which the examined product(s) are actively sold, i.e., the periods following the first non-zero demand observed for the series under evaluation.

スケール計算対象範囲の販売数0と、範囲外の販売数0を明確に区別する必要があるため、初めて商品が販売された日以前の販売数0をNaNに置換します。

In [ ]:
replacer = ReplaceBeforeFirstSell()
replacer.transform(sales, HISTORY_COLUMNS)
show_basic_info(sales)

### Total Sales

まず初めに、予測対象全体での販売数推移(日次)と移動平均(1週間,4週間,90日間)を可視化します。

グラフ右側の緑とピンクの色はそれぞれ、validationフェーズとevaluationフェーズの予測区間を表しています。

In [ ]:
x = calendar.iloc[:HISTORY_COUNTS, :]['date'].values
total_sales = sales[HISTORY_COLUMNS].sum()
fig = go.Figure(
    data=[go.Scatter(x=x, y=total_sales.values, name='raw')],
)
fig.add_trace(go.Scatter(x=x, y=total_sales.rolling(7).mean().values, name='1 week MA'))
fig.add_trace(go.Scatter(x=x, y=total_sales.rolling(28).mean().values, name='4 week MA'))
fig.add_trace(go.Scatter(x=x, y=total_sales.rolling(90).mean().values, name='90 days MA'))
fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='All item sales(sum)',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

グラフから読み取れる内容の例として、下記のようなことがわかります。

- 全体として緩やかな上昇トレンドが存在する。
- 12月24日はほとんどの商品販売を停止している。
- 1週間の中では週末に販売数が多く、週明けから半ばにかけて少ない。
- 年間では冬に販売数が少なく、夏にピークを迎える傾向にある。

### Mean Sales

学習期間の初期と終盤では、商品のユニーク数が異なることが分かっています。

予測対象としては個々の商品について予測することとなるため、1商品あたりの平均販売数の日次推移についても確認しておきましょう。

前処理によって、初めて販売数が非ゼロになるまでの商品販売数はNaNに置換されているためmean()メソッドによる平均計算対象からは省かれます。

In [ ]:
x = calendar.iloc[:HISTORY_COUNTS, :]['date'].values
total_sales = sales[HISTORY_COLUMNS].mean()
fig = go.Figure(
    data=[go.Scatter(x=x, y=total_sales.values, name='raw')],
)
fig.add_trace(go.Scatter(x=x, y=total_sales.rolling(7).mean().values, name='1 week MA'))
fig.add_trace(go.Scatter(x=x, y=total_sales.rolling(28).mean().values, name='4 week MA'))
fig.add_trace(go.Scatter(x=x, y=total_sales.rolling(90).mean().values, name='90 days MA'))
fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='All item sales(mean)',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

商品全体の売り上げとは対照的に、1商品あたりの販売数平均を見たときは下降傾向(ただし直近は横ばい)にあります。

このことから学習期間の初期と比較して、終盤はロングテールとなっていることがわかります。

### First sales

続いて上記の内容に関連して、予測対象(商品×店舗)のユニーク数の時系列推移をプロットします。

In [ ]:
date_map = {i + 1: datetime.datetime.strptime(calendar['date'].min(
), '%Y-%m-%d') + datetime.timedelta(days=i) for i in range(HISTORY_COUNTS)}

sales['first_sold'] = HISTORY_COUNTS - sales[HISTORY_COLUMNS].count(axis=1) + 1
sales['first_sold'] = sales['first_sold'].replace(date_map)

first_sales = sales.groupby('first_sold').size(
).reset_index().rename(columns={0: 'count'})
first_sales['cumulative count'] = first_sales['count'].cumsum()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=first_sales['first_sold'], y=first_sales['count'], mode='markers', name='First sold count'),
    secondary_y=False,)
fig.add_trace(go.Scatter(
    x=first_sales['first_sold'], y=first_sales['cumulative count'],
    name='Cumulative count', line = dict(color='firebrick', dash='dot')),
    secondary_y=True,)
fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')

fig.update_layout(
    title_text='Daily counts of item sold first time',
    xaxis_title='date',
)
fig.update_yaxes(title_text="Item count", secondary_y=False)
fig.update_yaxes(title_text="Cumulative item count", secondary_y=True)
fig.show()

- 最初の半年ほどで約半数の商品が販売され始め、以降は継続的に商品×店舗のユニーク数が増加している。
- 2016/02/17以降はNaNが存在せず、30490種類のID全ての販売数がRMSSEのscale計算に有効である。(validation期間の開始は2016/04/25～)

### Actual unique counts

販売が開始された商品が刻々と増え続けていることがわかりましたが、実際に販売数が1以上の商品のユニーク数は日々どのように推移しているのでしょうか。

このような観点から、30,490種類の予測対象系列について、販売数が1以上の系列の数の時系列推移をプロットします。

In [ ]:
x = calendar.iloc[:HISTORY_COUNTS, :]['date'].values
unique_sales = (sales[HISTORY_COLUMNS].fillna(0) > 0).sum()
fig = go.Figure(
    data=[go.Scatter(x=x, y=unique_sales.values, name='raw')],
)
fig.add_trace(go.Scatter(x=x, y=unique_sales.rolling(7).mean().values, name='1 week MA'))
fig.add_trace(go.Scatter(x=x, y=unique_sales.rolling(28).mean().values, name='4 week MA'))
fig.add_trace(go.Scatter(x=x, y=unique_sales.rolling(90).mean().values, name='90 days MA'))
fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='Unique Sales',
    xaxis_title='date',
    yaxis_title='unique sales count',
)
fig.show()

実際に販売されている商品の数も、右肩上がりであることがわかりました。

## Product Features

続いて商品関連の特徴量について、目的変数(sales)との相関を中心に調べます。

### Product kinds

まず最初に、最新時点での商品×店舗のユニーク数をcat_id毎、dept_id毎に確認します。

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

sales.groupby(['cat_id'])['item_id'].\
    count()[::-1].plot.barh(ax=ax1)

sales[sales['cat_id']=='FOODS'].\
    groupby(['dept_id'])['item_id'].\
    count()[::-1].plot.barh(ax=ax2)

sales[sales['cat_id']=='HOUSEHOLD'].\
    groupby(['dept_id'])['item_id'].\
    count()[::-1].plot.barh(ax=ax3)

sales[sales['cat_id']=='HOBBIES'].\
    groupby(['dept_id'])['item_id'].\
    count()[::-1].plot.barh(ax=ax4)

ax1.set_title('Item counts per category')
ax1.set_xlabel('item_count')
ax2.set_title('FOODS item counts per department')
ax2.set_xlabel('item_count')
ax3.set_title('HOUSEHOLDS item counts per department')
ax3.set_xlabel('item_count')
ax4.set_title('HOBBIES item counts per department')
ax4.set_xlabel('item_count')
fig.tight_layout()
plt.show()

- カテゴリ別の商品数としては、FOODS, HOUSEHOLD, HOBBIESの順に多い
- FOODSカテゴリの中でFOODS_3、HOBBIESカテゴリの中でHOBBIES_1の商品数の多さが目立つ。

上記と併せて、cat_id, dept_id毎の販売数合計も可視化してみます。

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

sales.groupby(['cat_id'])[HISTORY_COLUMNS].\
    sum().sum(axis=1)[::-1].plot.barh(ax=ax1)

sales[sales['cat_id']=='FOODS'].\
    groupby(['dept_id'])[HISTORY_COLUMNS].\
    sum().sum(axis=1)[::-1].plot.barh(ax=ax2)

sales[sales['cat_id']=='HOUSEHOLD'].\
    groupby(['dept_id'])[HISTORY_COLUMNS].\
    sum().sum(axis=1)[::-1].plot.barh(ax=ax3)

sales[sales['cat_id']=='HOBBIES'].\
    groupby(['dept_id'])[HISTORY_COLUMNS].\
    sum().sum(axis=1)[::-1].plot.barh(ax=ax4)

ax1.set_title('Item sales per category')
ax1.set_xlabel('item_count')
ax2.set_title('FOODS item counts per department')
ax2.set_xlabel('item_count')
ax3.set_title('HOUSEHOLDS item counts per department')
ax3.set_xlabel('item_count')
ax4.set_title('HOBBIES item counts per department')
ax4.set_xlabel('item_count')
fig.tight_layout()
plt.show()

ユニーク数のグラフとの比較により下記のようなことがわかります。

- HOUSEHOLD_1とHOUSEHOLD_2の間で、ユニーク数にそれほど差はないものの販売数に4倍近く差がある。
- HOUSEHOLDほどではないもののHOBBIESカテゴリにも同様の傾向あがる(ユニーク数と販売数の比率の違い)
- FOODSカテゴリは１アイテムあたりの販売数が多い傾向(特にFOODS_3)

### Sales per department

続いて、dept_id毎の日次販売数を確認します。

最初に1913日分の1商品あたりの販売数がどのように分布しているか、dept_id毎に大まかな特徴をつかむため、箱ひげ図で表します。(クリスマスは、外れ値として除外しています)

In [ ]:
dept_sales = sales.groupby(['dept_id'])[HISTORY_COLUMNS].mean().T
dept_sales = dept_sales.loc[calendar.loc[:HISTORY_COUNTS-1, :][~(calendar['event_name_1'] == 'Christmas')]['d'].values, :]

fig = go.Figure()
for col in dept_sales:
    fig.add_trace(go.Box(x=[col]*len(dept_sales), y=dept_sales[col], name=col))
    
fig.update_layout(
    title_text='Unit sales box plot per department',
    xaxis_title='dept_id',
    yaxis_title='mean sales per product',
)
fig.show()

箱ひげ図からは下記のような内容がわかります。

- FOOD_3カテゴリの販売数が、ほかのdept_idと比べて突出しており、日によって販売数の変動が激しいこともわかる。
- FOOD_1カテゴリとFOOD_2カテゴリでは中央値としては比較的近いが、FOOD_2の方が販売数の変動が激しい。
- どのカテゴリも共通して上側に外れ値が存在する。それぞれのdept_idについて日次販売数のヒストグラムを書くと、右裾の長い形状となることがわかる。

また、HOBBIES_2やHOUSEHOLD_2は基本的に1日の販売数が少ないことがわかるが、平均に対する外れ値の倍率が高い。今回の評価指標であるRMSSEは、販売数の変動が小さい商品の影響度を高くするようにスケール計算が行われるので、このような商品群に対する予測も重要であると考えられる。

In [ ]:
x = calendar.iloc[:HISTORY_COUNTS, :]['date'].values
dept_sales = sales.groupby(['dept_id'])[HISTORY_COLUMNS].sum()

fig = go.Figure()
for dept in dept_sales.index:
    y = dept_sales.loc[dept, :].values
    fig.add_trace(go.Scatter(x=x, y=y, name=dept))

fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='Item sales per department',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

In [ ]:
fig = go.Figure()
for dept in dept_sales.index:
    y = dept_sales.loc[dept, :].rolling(7).mean().values
    fig.add_trace(go.Scatter(x=x, y=y, name=dept))

fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='Item sales per department(1week MA)',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

箱ひげ図で読み取れた内容のいくつかは、上の時系列推移プロットからも読み取れます。時系列推移から読み取れる内容として、下記のようなことが挙げられます。

- dept_idをまたいで、基本的には販売数の上下は似たように推移していることがわかる。
- HOUSEHOLD_1の販売数が2012年6月に急増している。

### Weekday effect per department

これまで商品販売数の時系列推移をいくつかプロットしてきましたが、曜日周期性が存在することが明らかです。

そこでdept_id毎にどの曜日に販売されやすいか大まかにつかむための可視化を行います。

下記のヒートマップは全期間のdept_idの曜日ごとの販売数の割合を表しています。列(曜日)方向に和をとると1になります。

In [ ]:
dept_sales = sales.groupby(['dept_id'])[HISTORY_COLUMNS].sum().T
dept_sales_weekly = pd.merge(dept_sales, calendar[['d', 'weekday']], 
         left_index=True, right_on=['d']).groupby(['weekday']).sum()


fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
sns.heatmap(dept_sales_weekly.T.apply(lambda x: x / x.sum(), axis=1)
            [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']],
            cmap='Blues',
            annot=True,
            fmt='.3f',
            linewidths=.5)
ax.set_title('Weekday sales rate of each department', size=14)
plt.show()

- 全体的に週末に販売数が集中する。
- HOBBIES_2カテゴリは、比較的曜日により販売数のバラつきが少ない。
- FOOD_2カテゴリは金曜日より月曜日の販売数が多い唯一のカテゴリである。

### Products not recently sold

商品によっては、学習期間の前半には販売されていたものの販売が終了しているものも含まれる可能性があります。

そこで最新日からsalesが非ゼロの時点までを遡り、最新日時点で何日連続で販売がないかを確認してみましょう。(最新日が非ゼロの場合は0)

In [ ]:
df_values = sales[HISTORY_COLUMNS].values
tmp = np.tile(np.arange(1, len(HISTORY_COLUMNS) + 1)[::-1],
              (df_values.shape[0], 1))
tmp_values = ((df_values) > 0) * tmp
last_no = np.min(np.where(tmp_values == 0, 9999, tmp_values), axis=1) - 1
sales['last_sold'] = pd.to_datetime(calendar.loc[HISTORY_COUNTS - last_no - 1, 'date'].values)

last_sales = sales.groupby('last_sold').size(
).reset_index().rename(columns={0: 'count'})
last_sales['cumulative count'] = last_sales['count'].cumsum()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=last_sales['last_sold'], y=last_sales['count'], mode='markers', name='Last sold item count'),
    secondary_y=False,)
fig.add_trace(go.Scatter(
    x=last_sales['last_sold'], y=last_sales['cumulative count'],
    name='Cumulative count', line = dict(color='firebrick', dash='dot')),
    secondary_y=True,)
fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')

fig.update_layout(
    title_text='Daily counts of item sold last time',
    xaxis_title='date',
)
fig.update_yaxes(title_text="Item count", secondary_y=False)
fig.update_yaxes(title_text="Cumulative item count", secondary_y=True)
fig.show()

9割以上の商品は直近2週間の間に販売履歴が存在するが、少数ではあるが長い期間販売されていないものも存在する。

95%点と99%点を計算すると、それぞれ最終日である2016/04/24の1か月前・3か月半前程度となる。

In [ ]:
print(f"95%: {last_sales[(last_sales['cumulative count'] / NUM_ITEMS) <= 0.05]['last_sold'].max()}")
print(f"99%: {last_sales[(last_sales['cumulative count'] / NUM_ITEMS) <= 0.01]['last_sold'].max()}")

## Store Features

### Sales per store

store_id毎の日次販売数を確認します。

最初に1913日分の1商品あたりの販売数がどのように分布しているか、store_id毎に大まかな特徴をつかむため、箱ひげ図で表します。(同様にクリスマスは、外れ値として除外しています)

In [ ]:
store_sales = sales.groupby(['store_id'])[HISTORY_COLUMNS].mean().T
store_sales = store_sales.loc[calendar.loc[:HISTORY_COUNTS-1, :][~(calendar['event_name_1'] == 'Christmas')]['d'].values, :]

fig = go.Figure()
for col in store_sales:
    fig.add_trace(go.Box(x=[col]*len(store_sales), y=store_sales[col], name=col))
    
fig.update_layout(
    title_text='Unit sales box plot per store',
    xaxis_title='store_id',
    yaxis_title='mean sales per product',
)
fig.show()

- CA_3の販売数が他店舗と比較して突出している。
- TXの店舗の中で、TX_2は販売数平均のバラつきが大きいように見える
- 基本的にはどの店舗も共通して上側に外れ値が存在する。それぞれのstore_idについて日次販売数のヒストグラムを書くと、右裾の長い形状となることがわかる。
- WI_1とTX_2にはクリスマス以外で商品の販売が極端少ない日(下側の外れ値)が存在する。

続いて店舗毎の販売数の時系列推移をプロットします。

In [ ]:
x = calendar.iloc[:HISTORY_COUNTS, :]['date'].values
store_sales = sales.groupby(['store_id'])[HISTORY_COLUMNS].sum()

fig = go.Figure()
for store in store_sales.index:
    y = store_sales.loc[store, :].values
    fig.add_trace(go.Scatter(x=x, y=y, name=store))

fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='Item sales per store',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

In [ ]:
fig = go.Figure()
for store in store_sales.index:
    y = store_sales.loc[store, :].rolling(7).mean().values
    fig.add_trace(go.Scatter(x=x, y=y, name=store))

fig = plotly_util.add_range_selector(fig)
fig = plotly_util.add_shape_region(fig, '2016-04-25', '2016-05-23', 'LightSeaGreen')
fig = plotly_util.add_shape_region(fig, '2016-05-23', '2016-06-19', 'LightPink')
fig.update_layout(
    title_text='Item sales per store (1week MA)',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

- WI_1とWI_2ではそれぞれ2012年の10月と6月あたりに販売数が急増している。
- CA_2についても同様に2015年の6月から7月にかけて、販売数が急増している。
- 箱ひげ図からも同様に読み取れるように、時系列推移グラフにおいて、CA_4の販売数のバラつきが小さいことが目立つ。
- 1週間移動平均のグラフで1月に約2回のピークをもつ販売数の周期性がみられる

特にCA_2については比較的最近の2015年に販売数の急増があるため、その前後では販売数推移の傾向が変わっていることも考えられ、モデル学習の際に古いデータが予測に悪影響を及ぼさないか注意が必要である。

### Weekday effect per store

店舗の立地によって、どの曜日に販売されやすいかの傾向が異なることが考えられます。

商品カテゴリの場合と同様に店舗毎に曜日の販売数割合を大まかに把握するため、ヒートマップで可視化します。

In [ ]:
store_sales = sales.groupby(['store_id'])[HISTORY_COLUMNS].sum().T
store_sales_weekly = pd.merge(store_sales, calendar[['d', 'weekday']], 
         left_index=True, right_on=['d']).groupby(['weekday']).sum()


fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
sns.heatmap(store_sales_weekly.T.apply(lambda x: x / x.sum(), axis=1)
            [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']],
            cmap='Blues',
            annot=True,
            fmt='.3f',
            linewidths=.5)
ax.set_title('Weekday sales rate of each store', size=14)
plt.show()

- CA_2, WI_1の2店舗は特に週末に販売数が集中する傾向にある。
- CA_4, WI_2の2店舗は全店舗の中では満遍なく販売数が散らばる傾向にある。

## Other Features

### products × store raw data

同じ商品でも店舗によって客層が異なり、販売数が異なることも考えられます。

ここでもまずは、集計データではなく生データから確認してみましょう。ランダムにitem_idを選出し、各店舗での売り上げの推移を確認します。

In [ ]:
def random_plot(item_id=None):
    """
    item_idを引数に渡すことで描画するアイテムを直接指定することも可能
    """
    fig, axes = plt.subplots(5, 2, figsize=(15, 10))
    if item_id is None:
        item = random.sample(list(sales['item_id'].unique()), 1)[0]
    else:
        item = item_id

    x = pd.to_datetime(calendar.iloc[:HISTORY_COUNTS, :]['date']).values
    for ax, store in zip(axes.flatten(), sales['store_id'].unique()):
        y = sales[(sales['item_id']==item) & (sales['store_id']==store)][HISTORY_COLUMNS].values[0]
        ax.plot(x, y)
        ax.set_title(store)
        ax.set_xlabel('date')
        ax.set_ylabel('sales')

    fig.suptitle(f'{item} sales per store', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
random_plot('FOODS_3_090')

In [ ]:
random_plot('HOUSEHOLD_1_528')

In [ ]:
random_plot('FOODS_2_243')

商品をランダムに選出し、店舗ごとの販売数の時系列推移を数回比較する。(例示のため、直接item_idを指定してるが引数にitem_idを指定せずに実行するとランダムに商品が選ばれる)

- 商品を販売休止している期間が存在し、店舗毎に異なる場合がある(FOODS_2_243)
- 販売数の多いFOODS_3_090のように、同タイミングで販売数が0に落ち込む商品も存在する。

生データを確認することで、多様な推移が存在し予測が容易ではないことがわかる。

### department rate per store

続いて、店舗毎にどのdept_idの商品が売れやすいかを大まかに把握するために店舗毎のdept_idの販売数比率をヒートマップで可視化する。

In [ ]:
store_dept_sales = sales.groupby(['store_id', 'dept_id']).sum().sum(axis=1).unstack()
store_dept_sales = store_dept_sales.apply(lambda x: x / x.sum(), axis=1)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
sns.heatmap(store_dept_sales,
            cmap='RdPu',
            annot=True,
            fmt='.3f',
            linewidths=.5)
ax.set_title('Department sales rate of each store', size=14)
plt.show()

- 全般的にFOODS_3の販売数が占める割合が大きい。FOODS_3が全体の半数以上である店舗が3店舗(CA_1, TX_2, WI_3)
- その中で、CA_2はFOODS_3の販売数が占める割合が比較的小さく、FOODS_1の占める割合が他の店舗と比較して大きい。
- その他でもHOBBIES_1やHOUSEHOLD_1は店舗によって、割合の差が大きいことが様子が見て取れる。

### SNAP effect

続いてSNAP購買の可否は、販売数にどの程度の影響を及ぼしているかといった観点での可視化を行います。

まずはじめに、学習期間における各州のSNAPの比率を確認します。

In [ ]:
snap_ca = calendar.iloc[:HISTORY_COUNTS, :][['date', 'weekday', 'snap_CA']].set_index('date')
snap_tx = calendar.iloc[:HISTORY_COUNTS, :][['date', 'weekday', 'snap_TX']].set_index('date')
snap_wi = calendar.iloc[:HISTORY_COUNTS, :][['date', 'weekday', 'snap_WI']].set_index('date')

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4))
snap_ca['snap_CA'].value_counts().plot.bar(ax=ax1)
snap_tx['snap_TX'].value_counts().plot.bar(ax=ax2)
snap_wi['snap_WI'].value_counts().plot.bar(ax=ax3)

ax1.set_title('SNAP days in CA')
ax2.set_title('SNAP days in TX')
ax3.set_title('SNAP days in WI')
ax1.set_xticklabels(['No SNAP', 'SNAP'])
ax2.set_xticklabels(['No SNAP', 'SNAP'])
ax3.set_xticklabels(['No SNAP', 'SNAP'])
ax1.set_ylabel('Day counts')
ax2.set_ylabel('Day counts')
ax3.set_ylabel('Day counts')
plt.tight_layout()

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)

calendar.iloc[:HISTORY_COUNTS, :][['date', 'weekday', 'snap_CA', 'snap_TX', 'snap_WI']].\
    set_index('date').groupby('weekday').sum().plot.barh(ax=ax)
ax.set_title('SNAP day counts per weekday')

plt.show()

各州についてSNAP購買が許可される日付の合計数は同じですが、日付は異なることがあります。(曜日ごとのSNAP購買が許可された日数の違いとして表れています)

続いて各州について、SNAPとNo SNAPの場合に分けて販売数の時系列推移を確認してみます。

In [ ]:
cax1 = snap_ca[snap_ca['snap_CA']==1].reset_index()['date'].values
cay1 = np.extract((snap_ca['snap_CA']==1).values,
                sales[sales['state_id']=='CA'][HISTORY_COLUMNS].sum().values)

cax2 = snap_ca[snap_ca['snap_CA']==0].reset_index()['date'].values
cay2 = np.extract((snap_ca['snap_CA']==0).values,
                sales[sales['state_id']=='CA'][HISTORY_COLUMNS].sum().values)

txx1 = snap_tx[snap_tx['snap_TX']==1].reset_index()['date'].values
txy1 = np.extract((snap_tx['snap_TX']==1).values,
                sales[sales['state_id']=='TX'][HISTORY_COLUMNS].sum().values)

txx2 = snap_tx[snap_tx['snap_TX']==0].reset_index()['date'].values
txy2 = np.extract((snap_tx['snap_TX']==0).values,
                sales[sales['state_id']=='TX'][HISTORY_COLUMNS].sum().values)

wix1 = snap_wi[snap_wi['snap_WI']==1].reset_index()['date'].values
wiy1 = np.extract((snap_wi['snap_WI']==1).values,
                sales[sales['state_id']=='WI'][HISTORY_COLUMNS].sum().values)

wix2 = snap_wi[snap_wi['snap_WI']==0].reset_index()['date'].values
wiy2 = np.extract((snap_wi['snap_WI']==0).values,
                sales[sales['state_id']=='WI'][HISTORY_COLUMNS].sum().values)


fig = make_subplots(rows=3, cols=1, subplot_titles=('Item Sales in CA', 'Item Sales in TX', 'Item Sales in WI'))
fig.add_trace(go.Scatter(x=cax1, y=cay1, name='SNAP(CA)'), row=1, col=1)
fig.add_trace(go.Scatter(x=cax2, y=cay2, name='No SNAP(CA)'), row=1, col=1)
fig.add_trace(go.Scatter(x=txx1, y=txy1, name='SNAP(TX)'), row=2, col=1)
fig.add_trace(go.Scatter(x=txx2, y=txy2, name='No SNAP(TX)'), row=2, col=1)
fig.add_trace(go.Scatter(x=wix1, y=wiy1, name='SNAP(WI)'), row=3, col=1)
fig.add_trace(go.Scatter(x=wix2, y=wiy2, name='No SNAP(WI)'), row=3, col=1)

fig = plotly_util.add_range_selector(fig)
fig.update_layout(
    height=1000,
    title_text='Item sales by SNAP and No SNAP',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

SNAPフラグが1の日と0の日の販売数推移を州別でプロットしたところ下記のようなことがわかる。

- どの州もSNAPフラグが1の日の方が、平均的に販売数が高いことがわかる。
- 中でもWI州はプロットを見て一目でわかる程度に、他の州よりもSNAPの効果が大きい。

SNAPによる効果の大きさは、WI>>TX>CAであるがSNAPの日の平均販売数とNo SNAPの日の平均販売数の比率を確認する以下のグラフでも確認できる。

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=True,
                    subplot_titles=(f'CA ({cay1.mean() / cay2.mean():.3f})',
                                    f'TX ({txy1.mean() / txy2.mean():.3f})',
                                    f'WI ({wiy1.mean() / wiy2.mean():.3f})'))

fig.add_trace(go.Bar(x=['SNAP', 'No SNAP'], y=[cay1.mean(), cay2.mean()], name='CA'), row=1, col=1)
fig.add_trace(go.Bar(x=['SNAP', 'No SNAP'], y=[txy1.mean(), txy2.mean()], name='TX'), row=1, col=2)
fig.add_trace(go.Bar(x=['SNAP', 'No SNAP'], y=[wiy1.mean(), wiy2.mean()], name='WI'), row=1, col=3)

fig.update_layout(
    height=400,
    title_text='Item average daily sales by SNAP and No SNAP',
    xaxis_title='date',
    yaxis_title='sales',
)
fig.show()

SNAPは貧しい家族向けの制度であり、SNAP対象外の家庭には無関係であるため、貧しい家庭の多い地域の店舗で販売数増加の効果が大きいと考えられます。

### Event calendar in 2015

直近の1年について、どのようなイベントがあるかを総販売数グラフにアノテーションをつける形で確認します。

2016年は4月までしかデータが存在しないので、2015年のイベントを表示します。

In [ ]:
fig = make_subplots(rows=6, cols=2)
total_sales = sales[HISTORY_COLUMNS].sum()

for month in range(1, 13):
    x = calendar[(calendar['year'] == 2015) & (calendar['month'] == month)]['date'].values
    events = calendar[(calendar['event_name_1'].notnull()) &
                      (calendar['year'] == 2015) &
                      (calendar['month'] == month)]
    fig = fig.add_trace(go.Scatter(
        x=x,
        y=total_sales.loc[
            calendar[(calendar['year'] == 2015) & (calendar['month'] == month)]['d']
            ].values,
        name=f'month: {month}',
        ),
        row=(month - 1) // 2 + 1,
        col=(month - 1) % 2 + 1,
        )
    for idx in events.index:
        fig.add_annotation(x=events.loc[idx, 'date'],
                           y=total_sales.loc[events.loc[idx, 'd']],
                           text=events.loc[idx, 'event_name_1'],
                           row=(month - 1) // 2 + 1,
                           col=(month - 1) % 2 + 1)
# fig = plotly_util.format_annotation(fig)
fig.update_layout(
    height=1200,
    title_text='Item Sales in 2015 with events',
)
fig.show()

クリスマスには販売を停止していることは、これまでの分析で確認した通りですが、それ以外のイベントでは総販売数で見て一目でわかるほどのイベント効果はないようです。(あくまで目視レベルでの話です。7月のIndependenceDayは週末の販売数増加に逆らって減少している感じがします。)

## Basic Timeseries Analysis

**TODO**

## Learn  about RMSSE(WRMSSE)

これまでcsv形式で与えられたデータセットから様々な集計を実施してきました。

本コンペティションでは、モデルの予測の良さを評価するための評価指標が定められているため、モデリングに進む前に評価指標と指標の選択理由についてしっかりと把握しておきたいと思います。

**TODO**

## Proceed to FE and modeling

ここまでの内容をもとに、特徴量エンジニアリングとモデリングに進みます。

またリーダーボードのPublicスコアに最適化しないように、良いCV方法の検討も併せて実施します。

M5_Competition_FE_and_modelingカーネルにて公開予定。